<a href="https://colab.research.google.com/github/abulhasanat/MachineLearning/blob/master/Web_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install JPype1

     |████████████████████████████████| 2.7MB 2.9MB/s 


In [0]:
!git clone https://github.com/misja/python-boilerpipe.git

Cloning into 'python-boilerpipe'...
remote: Enumerating objects: 264, done.
remote: Total 264 (delta 0), reused 0 (delta 0), pack-reused 264
Receiving objects: 100% (264/264), 30.28 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [0]:
%cd /content/python-boilerpipe
!sudo python setup.py install

/content/python-boilerpipe
/usr/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/boilerpipe
copying src/boilerpipe/__init__.py -> build/lib/boilerpipe
creating build/lib/boilerpipe/extract
copying src/boilerpipe/extract/__init__.py -> build/lib/boilerpipe/extract
creating build/lib/boilerpipe/data
creating build/lib/boilerpipe/data/boilerpipe-1.2.0
copying src/boilerpipe/data/boilerpipe-1.2.0/boilerpipe-1.2.0.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0
creating build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
copying src/boilerpipe/data/boilerpipe-1.2.0/lib/xerces-2.9.1.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
copying src/boilerpipe/data/boilerpipe-1.2.0/lib/nekohtml-1.9.13.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
running install_lib
creating /usr/local/lib/python3.6/dist-packages/boilerpi

In [0]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import numpy as np
import re
from urllib.parse import urlparse

In [0]:
import requests
requests.packages.urllib3.disable_warnings()

import random
import math
import string
import re
import numpy as np
import nltk
nltk.download('punkt')
import networkx as nx
import json

import collections
from boilerpipe.extract import Extractor

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [0]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [0]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [0]:
%%bash
pip install pytorch-pretrained-bert

  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609182 sha256=b33783e16aba1cd4763a78b280381417a9e537125cfe531062efb0281bd7d13a
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=True,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=True,
    bs=32,
    discriminative=False,
    max_seq_len=256,
)

In [0]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

100%|██████████| 231508/231508 [00:00<00:00, 6423798.32B/s]


In [0]:
def _join_texts(texts:Collection[str], mark_fields:bool=False, sos_token:Optional[str]=BOS):
    """Borrowed from fast.ai source"""
    if not isinstance(texts, np.ndarray): texts = np.array(texts)
    if is1d(texts): texts = texts[:,None]
    df = pd.DataFrame({i:texts[:,i] for i in range(texts.shape[1])})
    text_col = f'{FLD} {1} ' + df[0].astype(str) if mark_fields else df[0].astype(str)
    if sos_token is not None: text_col = f"{sos_token} " + text_col
    for i in range(1,len(df.columns)):
        #text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i]
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i].astype(str)
    return text_col.values

In [0]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),NumericalizeProcessor(vocab=vocab)]

In [0]:
class BertDataBunch(TextDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                tokenizer:Tokenizer=None, vocab:Vocab=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames."
        p_kwargs, kwargs = split_kwargs_by_func(kwargs, get_bert_processor)
        # use our custom processors while taking tokenizer and vocab as kwargs
        processor = get_bert_processor(tokenizer=tokenizer, vocab=vocab, **p_kwargs)
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() if cls==TextLMDataBunch else src.label_from_df(cols=label_cols, classes=classes)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        return src.databunch(**kwargs)

In [0]:
def getUA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]

    return random.choice(uastrings)

def prepare_sentences(text):
    text=text.replace('\n',' ')
    # Clean characters
    text = "".join([t for t in text if t.isalnum() or t in string.punctuation or t == ' '])
    
    # Get rid of whitespace characters
    sentences = nltk.sent_tokenize(text)
    text = " ".join([s.strip() for s in sentences])
    
    # Fix puctuation spacing
    text = re.sub(r"(\w{2,})([\.\!\?]+)(\w)", r"\1\2 \3", text)
    text = re.sub(' +', ' ', text)
    return text

def find_children(n1, n2):
    
    children1 = [c for c in n1.children]
    children2 = [c for c in n2.children]
    
    for i, children in enumerate([children1,children2]):
        level = 1

        while children:
            lchildren = children
            children = []
            for ch in lchildren:
                if not ch.name:
                    continue
                if ch == n1 or ch == n2:
                    if i == 0:
                        return len( nltk.sent_tokenize(n2.get_text()) ) * ( 1/math.exp(level) )
                    else:
                        return len( nltk.sent_tokenize(n1.get_text()) ) * ( 1/math.exp(level) )
                                        
                children.extend(ch.children)

            level += 1
            
    return 0

def extract_html(url, timeout=10):
    headers = {'user-agent': getUA()}
    r = requests.get(url, headers = headers, verify=False, timeout=timeout)
    html = r.content
    return html

def tokenize_url(url:str):   
    url=re.sub("(\W|_)+"," ",url)
    return url

In [0]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [0]:
def get_title(soup):
    try:
        for i in list(soup.find_all('title')):
            return(i.get_text().replace("\n"," ").replace("\t"," ").strip())
    except:
        pass

def list2string(x):
    s=""
    try:
        for i in x:
            for j in i:
                if j!=None:
                    s = s + j+" "
    except:
        pass
    return (s)

def get_metadata_property(soup):
    try:
        x=[i.get('property') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[x,v]
        return list2string(z)
    except:
        pass

def get_metadata_name(soup):
    try:
        y=[i.get('name') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[y,v]
        return list2string(z)
    except:
        pass

def get_metadata_content(soup):
    try:
        v=[]
        v=[i.get('content') for i in soup.findAll('meta')]
    #     return list2string(v)
        return (v)
    except:
        pass

def get_text(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return text
#     return (len(text.split())/COUNT)

In [0]:
def get_word_count(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return len(text.split())

def get_para_count(soup):
    count=0
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            count+=1
    except:
        pass
    return count

In [0]:
def get_internal_links(url_main,links):
    internal_url_list=[]
    try:
        for item in links:
            for url_parts in url_main.split('.')[:-1]:
                if (url_parts in item.lower()):
                    internal_url_list.append(item)
        internal_url_list=list(dict.fromkeys(internal_url_list))    
        return(len(internal_url_list))
    except:
        pass

def get_external_links(url_main,links):
    external_url_list=[]
    try:
        for item in links:
            count=0
            for url_parts in url_main.split('.')[:-1]:
                if (item.startswith(('http', 'ftp', 'www')) and (url_parts not in item.lower())):
                    count+=1
            if count==len(url_main.split('.')[:-1]):
                external_url_list.append(item)
        return(len(external_url_list))
    except:
        pass

In [0]:
def extract_features(url):
    # create a new Chrome session
    try:
        print(url)
        metadata_property=''
        metadata_name=''
        metadata_content=''
        title=''
        word_count=0
        sen_count=0
        internal_links=0
        external_links=0
    #         driver = webdriver.Chrome(executable_path=r'D:/chromedriver_win32/chromedriver.exe')
    #         driver.set_page_load_timeout(60)
    #         driver.get(url)
    #         html = driver.page_source
    #         driver.close()
    #         soup = BeautifulSoup(html, 'html.parser')  #Parse html code       
    #         body = soup.find('body')

        html= extract_html(url, 100)
        soup = BeautifulSoup(html, 'lxml')
        body = soup.find('body')
        remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                             "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
        text = ""
        boilerpipe_sentences=""
        boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
        boilerpipe_sentences = prepare_sentences(boilerpipe_content)
    #         print('Sentences:',len(boilerpipe_sentences))

#         p = '(?:http.*://)?(?:www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        p = '(?:http.*://)?(?:www[0-9].|www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        m = re.search(p,url)
        url_main = m.group('host') # 'www.abc.com'



    #         return pd.Series([get_metadata_property(soup),get_metadata_name(soup),get_metadata_content(soup),
    #                           get_title(soup),len(nltk.word_tokenize(boilerpipe_sentences)),
    #                           len(nltk.sent_tokenize(boilerpipe_sentences)),
    #                           get_internal_links(url_main,links),
    #                           get_external_links(url_main,links)])

        print(len(soup))
        if len(soup)>0:
            metadata_property=get_metadata_property(soup)
            metadata_name=get_metadata_name(soup)
            metadata_content=get_metadata_content(soup)
            title=get_title(soup)
            
            word_count=len(nltk.word_tokenize(boilerpipe_sentences))
            
            sen_count=len(nltk.sent_tokenize(boilerpipe_sentences))       

            links = [ x['href'] for x in soup.find_all('a', href=True) ]
            if len(links)>0:
                internal_links=get_internal_links(url_main,links)
                external_links=get_external_links(url_main,links)

    #         return pd.Series([get_metadata_property(soup),get_metadata_name(soup),get_metadata_content(soup),
    #                       get_title(soup)])
    except:
        pass
    
    extract= pd.Series([metadata_property,metadata_name,metadata_content,
                          title,word_count,sen_count,internal_links,external_links])
    #         extract= pd.Series([get_metadata_property(soup),get_metadata_name(soup)])
    print(extract)
    return extract

In [0]:
# test=pd.DataFrame(pd.Series([boilerpipe_sentences,0,0,0,0,0,0]),columns=['category','label_1',	'label_3',	'label_5',	'label_6',	'label_7',	'label_8'])

In [0]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), pre_rules=[], post_rules=[])

In [0]:
sample=False
DATA_ROOT=Path('/content/drive/My Drive/Data/DMOZ')
df = pd.read_csv(DATA_ROOT / "train.csv")
try:
  df.drop(['Unnamed: 0'],axis=1,inplace=True)
except:
  pass
df.dropna(inplace=True)
df['cat_id']=df['cat_id'].astype(int)
if sample==True:
  df_sample=df[df['cat_id']<9]
  train=pd.concat([df_sample,pd.get_dummies(df_sample['cat_id'], prefix='label')],axis=1)
  df=df_sample
else:  
  # train=df 
  train=pd.concat([df,pd.get_dummies(df['cat_id'], prefix='label')],axis=1) 
val=train

del df

In [0]:
train.sample(1)

In [0]:
# label_cols=['label_1',	'label_3',	'label_5',	'label_6',	'label_7',	'label_8']

In [0]:
url='https://www.amazon.com/'
html= extract_html(url, 60)
soup = BeautifulSoup(html, 'lxml')
body = soup.find('body')
# remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
#                                      "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                     "svg", "xml"])]
boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
boilerpipe_sentences = prepare_sentences(boilerpipe_content)
nltk.sent_tokenize(boilerpipe_sentences)

['Try Prime Deliver toMountainview 94035 12 Days of Deals Best Sellers Find a Gift Customer Service New Releases Registry Gift Cards Sell AmazonBasics Coupons Whole Foods Free Shipping Shopper Toolkit #FoundItOnAmazon Welcome to Amazon.',
 'com.',
 'If you prefer a simplified shopping experience, try the mobile web version of Amazon at www.',
 'amazon.',
 'com/access.',
 'The mobile web version is similar to the mobile app.',
 'Stay on Amazon.',
 'com for access to all the features of the main Amazon website.',
 'Learn more about Amazon Prime.',
 'Prime members enjoy FREE Two-Day Delivery and exclusive access to music, movies, TV shows, original audio series, and Kindle books.',
 'View or edit your browsing history After viewing product detail pages, look here to find an easy way to navigate back to pages you are interested in.']

In [0]:
# num_labels=6
label_cols=[]
num_labels=train['cat_id'].nunique()

data={'cat_id':[0],
      'category':[boilerpipe_sentences],
}
test=pd.DataFrame(data)
for labels in train['cat_id'].unique():
  label='label_'+str(labels)
  label_cols.append(label)
  test[label]=0

test=test.append(test)
test

In [0]:
# train=train.head(5)
# val=train
databunch = BertDataBunch.from_df(".", train, val, test,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  text_cols="category",
                  label_cols=label_cols,
                  bs=config.bs,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

In [0]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=num_labels)

In [0]:
loss_func = nn.BCEWithLogitsLoss()

In [0]:
from fastai.callbacks import *
device_cuda = torch.device("cuda")
learner = Learner(
    databunch, bert_model,
    loss_func=loss_func
)
# if config.use_fp16: learner = learner.to_fp16()
learner.to_fp32()
learner.load(file=DATA_ROOT/'Model_DMOZ(6_cat).pkl')

In [0]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in databunch.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

In [0]:
test_preds = get_preds_as_nparray(DatasetType.Test)

In [0]:
sample_submission=test
for label in label_cols:
  sample_submission[label]=0
sample_submission[label_cols] = test_preds
# sample_submission.to_csv(DATA_ROOT /"predictions_1.csv", index=False)
# sample_submission.sample(10000).to_csv(DATA_ROOT /"predictions_sample.csv", index=False)

In [0]:
sample_submission

In [0]:
output=sample_submission.head(1).drop(columns=['cat_id','category']).T
output

In [0]:
output[0]=output[0].astype(float)
output.nlargest(3,[0])

In [0]:
DATA_ROOT=Path('/content/drive/My Drive/Data/DMOZ')
# learner.load(file=DATA_ROOT/'Model_DMOZ(6_cat).pkl')
learn = load_learner(DATA_ROOT, 'Model_DMOZ(6_cat).pkl')


In [0]:
learn.to_fp32()
learn.predict(boilerpipe_sentences)

(MultiCategory label_3,
 tensor([0., 1., 0., 0., 0., 0.]),
 tensor([0.0496, 0.7276, 0.1668, 0.0226, 0.0220, 0.0204]))